In [1]:
import requests
from secret import ACCESS_KEY
import pandas as pd
from collections import defaultdict
from datetime import datetime
import calendar

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)



### This is the API call for "Private Residential Property Transactions"

In [2]:
headers = {
    "AccessKey": ACCESS_KEY
}
resp = requests.get('https://www.ura.gov.sg/uraDataService/insertNewToken.action', headers=headers)
token = resp.json()['Result']
token

'8+6Xax@2KAK8xyd9d94fd66btwU6610EZ1b8jt78p7fr1Q1MdsY6v2186nV@9Ft7bJ7--02aeweff7t071-NfY0sbNH2dmSE2t7-'

In [3]:
headers = {
    'AccessKey': ACCESS_KEY,
    'Token': token
}
params = {
    'service': 'PMI_Resi_Transaction',
    'batch': 3,
}
resp = requests.get('https://www.ura.gov.sg/uraDataService/invokeUraDS', headers=headers, params=params)
resp

<Response [200]>

In [270]:
def convert_str_to_date(input_str):
    """ converting dates of 0220 for example to 29-02-2020"""
    dt = datetime.strptime(input_str, '%m%y')
    last_day = calendar.monthrange(dt.year, dt.month)[1]
    dt = dt.replace(day=last_day)
    return dt.strftime('%Y-%m-%d')

def map_type_of_sale(type_of_sale_val):
    mapping = {
        '1': 'New Sale',
        '2': 'Sub Sale',
        '3': 'Resale'
    }
    return mapping.get(type_of_sale_val)

res = defaultdict(list)
df = pd.DataFrame()
for item in resp.json()['Result']:
    for trans in item['transaction']:
        res['y'].append(round(float(item['y']), 12) if item.get('y') else None)
        res['x'].append(round(float(item['x']), 12) if item.get('x') else None)
        res['street'].append(item['street'])
        res['project'].append(item['project'])

        res['market_segment'].append(item['marketSegment'])
        
        area = float(trans['area'])
        area_sqft = area * 10.76391042
        res['area_sqm'].append(area)
        res['area_sqft'].append(area_sqft)
        
        res['price'].append(float(trans['price']))
        res['nett_price'].append(float(trans['nettPrice']) if trans.get('nettPrice') else None)
        
        res['unit_price_psm'].append(float(trans['price']) / area)
        res['unit_price_psf'].append(float(trans['price']) / area_sqft)
        
        res['floor_range'].append(trans['floorRange'])
        res['num_units'].append(float(trans['noOfUnits']))
        res['reference_period'].append(convert_str_to_date(trans['contractDate']))
        res['type_of_sale'].append(map_type_of_sale(trans['typeOfSale']))
        res['property_type'].append(trans['propertyType'])
        res['district'].append(int(trans['district']))
        res['type_of_area'].append(trans['typeOfArea'])
        res['tenure'].append(trans['tenure'])
df = df.append(pd.DataFrame(res))
df

,y,x,street,project,market_segment,area_sqm,area_sqft,price,nett_price,unit_price_psm,unit_price_psf,floor_range,num_units,reference_period,type_of_sale,property_type,district,type_of_area,tenure
0,32612.843992,38874.764492,ELLIOT ROAD,ELLIOT AT THE EAST COAST,OCR,122.0,1313.197071,1880000.0,NaN,15409.836066,1431.620616,01-05,1.0,2019-04-30,Resale,Condominium,15,Strata,Freehold
1,32612.843992,38874.764492,ELLIOT ROAD,ELLIOT AT THE EAST COAST,OCR,137.0,1474.655728,1900000.0,NaN,13868.613139,1288.436321,01-05,1.0,2017-08-31,Resale,Condominium,15,Strata,Freehold
2,32612.843992,38874.764492,ELLIOT ROAD,ELLIOT AT THE EAST COAST,OCR,199.0,2142.018174,2450000.0,NaN,12311.557789,1143.781145,01-05,1.0,2018-09-30,Resale,Condominium,15,Strata,Freehold
3,32612.843992,38874.764492,ELLIOT ROAD,ELLIOT AT THE EAST COAST,OCR,161.0,1732.989578,2050000.0,NaN,12732.919255,1182.926906,01-05,1.0,2017-08-31,Resale,Condominium,15,Strata,Freehold
4,32612.843992,38874.764492,ELLIOT ROAD,ELLIOT AT THE EAST COAST,OCR,47.0,505.903790,800000.0,NaN,17021.276596,1581.328340,01-05,1.0,2018-12-31,Resale,Condominium,15,Strata,Freehold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28800,42560.406000,35969.634000,PUNGGOL FIELD,PRIVE,OCR,98.0,1054.863221,1080000.0,NaN,11020.408163,1023.829420,06-10,1.0,2019-10-31,Resale,Executive Condominium,19,Strata,99 yrs lease commencing from 2010
28801,42560.406000,35969.634000,PUNGGOL FIELD,PRIVE,OCR,102.0,1097.918863,1080000.0,NaN,10588.235294,983.679247,11-15,1.0,2018-08-31,Resale,Executive Condominium,19,Strata,99 yrs lease commencing from 2010
28802,42560.406000,35969.634000,PUNGGOL FIELD,PRIVE,OCR,102.0,1097.918863,1065000.0,NaN,10441.176471,970.017035,11-15,1.0,2018-07-31,Resale,Executive Condominium,19,Strata,99 yrs lease commencing from 2010
28803,42560.406000,35969.634000,PUNGGOL FIELD,PRIVE,OCR,102.0,1097.918863,1058888.0,NaN,10381.254902,964.450139,11-15,1.0,2018-08-31,Resale,Executive Condominium,19,Strata,99 yrs lease commencing from 2010


In [288]:
df1 = pd.read_csv('data/private_prop_transactions/total_transactions.csv', float_precision='round_trip')
# df1 = df1.drop_duplicates(ignore_index=True)
x = df1.sort_values(['project', 'reference_period', 'area_sqft']).iloc[[0]].reset_index(drop=True)
y = df1.sort_values(['project', 'reference_period', 'area_sqft']).iloc[[1]].reset_index(drop=True)

df1[df1.duplicated(subset=['street', 'project', 'market_segment', 'area_sqm', 'price', 'nett_price', 'floor_range', 'num_units', 'reference_period', 'type_of_sale', 'property_type', 'district', 'type_of_area', 'tenure'], keep=False)]
# assert_frame_equal(x, y, check_exact=True, check_index_type=True, check_datetimelike_compat=True, check_categorical=True)
# df1.drop_duplicates(subset=['street', 'project', 'market_segment', 'area_sqm', 'price', 'floor_range', 'num_units', 'reference_period', 'type_of_sale', 'property_type', 'district', 'type_of_area', 'tenure'])

,y,x,street,project,market_segment,area_sqm,area_sqft,price,nett_price,unit_price_psm,unit_price_psf,floor_range,num_units,reference_period,type_of_sale,property_type,district,type_of_area,tenure


In [282]:
df2 = df1.append(df)
df3 = df2.drop_duplicates(subset=['street', 'project', 'market_segment', 'area_sqm', 'price', 'floor_range', 'num_units', 'reference_period', 'type_of_sale', 'property_type', 'district', 'type_of_area', 'tenure'], ignore_index=True)
# x = df3[df3['project'] == 'ZEPHYR PARK'].sort_values(['reference_period', 'area_sqft']).iloc[[0]].reset_index(drop=True)
# y = df3[df3['project'] == 'ZEPHYR PARK'].sort_values(['reference_period', 'area_sqft']).iloc[[1]].reset_index(drop=True)
# df3[df3['project'] == 'ELLIOT AT THE EAST COAST'].sort_values(['reference_period', 'area_sqft'])
from pandas._testing import assert_frame_equal
assert_frame_equal(x,y)
# df3[df3['project'] == 'ZEPHYR PARK'].sort_values(['reference_period', 'area_sqft'])
# df3.sort_values(['project', 'reference_period', 'area_sqft'])

# assert_frame_equal(x,y, check_exact=True)
df3

,y,x,street,project,market_segment,area_sqm,area_sqft,price,nett_price,unit_price_psm,unit_price_psf,floor_range,num_units,reference_period,type_of_sale,property_type,district,type_of_area,tenure
0,29491.915576,22726.561999,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314.0,3379.867872,4750000.0,NaN,15127.388535,1405.380382,-,1.0,2018-06-30,Resale,Semi-detached,5,Land,Freehold
1,29491.915576,22726.561999,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308.0,3315.284409,5000000.0,NaN,16233.766234,1508.166233,-,1.0,2018-09-30,Resale,Semi-detached,5,Land,Freehold
2,29491.915576,22726.561999,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,5643.518233,5500000.0,NaN,10490.177379,974.569368,-,1.0,2020-02-29,Resale,Semi-detached,5,Land,Freehold
3,25005.804519,28384.068816,COVE DRIVE,TURQUOISE,CCR,348.0,3745.840826,4400000.0,NaN,12643.678161,1174.636138,06-10,1.0,2018-09-30,Resale,Condominium,4,Strata,99 yrs lease commencing from 2007
4,25005.804519,28384.068816,COVE DRIVE,TURQUOISE,CCR,224.0,2411.115934,3600000.0,NaN,16071.428571,1493.084571,01-05,1.0,2018-03-31,Resale,Condominium,4,Strata,99 yrs lease commencing from 2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67394,45396.257688,27100.634621,GOODLINK PARK,GOODLINK PARK,OCR,268.2,2886.880775,2480000.0,NaN,9246.830723,859.058684,-,1.0,2017-10-31,Resale,Terrace,27,Land,Freehold
67395,32699.327274,36157.514620,LORONG NANGKA,LANDED HOUSING DEVELOPMENT,RCR,184.4,1984.865081,2750000.0,NaN,14913.232104,1385.484598,-,1.0,2017-08-31,Resale,Terrace,15,Land,NA
67396,36761.646727,29104.810081,CLOVER RISE,LANDED HOUSING DEVELOPMENT,RCR,167.4,1801.878604,2480000.0,NaN,14814.814815,1376.341333,-,1.0,2017-08-31,Resale,Terrace,20,Land,NA
67397,32671.914414,34973.777626,IPOH LANE,EMERY POINT,RCR,121.0,1302.433161,725000.0,NaN,5991.735537,556.650446,01-05,1.0,2017-07-31,Resale,Apartment,15,Strata,NA


In [272]:
# df['property_type'].unique()
df3[df3['project'] == 'ZEPHYR PARK'].sort_values(['reference_period', 'area_sqft']).iloc[[0, 1]]

,y,x,street,project,market_segment,area_sqm,area_sqft,price,nett_price,unit_price_psm,unit_price_psf,floor_range,num_units,reference_period,type_of_sale,property_type,district,type_of_area,tenure
36734,35592.978832,41180.67381,SEA BREEZE ROAD,ZEPHYR PARK,OCR,269.6,2901.950249,2980000.0,NaN,11053.412463,1026.89562,-,1.0,2017-11-30,Resale,Terrace,16,Land,Freehold
74075,35592.978832,41180.67381,SEA BREEZE ROAD,ZEPHYR PARK,OCR,269.6,2901.950249,2980000.0,NaN,11053.412463,1026.89562,-,1.0,2017-11-30,Resale,Terrace,16,Land,Freehold


In [133]:
assert x.at[0, 'type_of_sale'] == y.at[0, 'type_of_sale']

In [277]:
x.at[0, 'x']
# x.dtypes

41180.673810069995

In [278]:
y.at[0, 'x']
# y.dtypes

41180.673810069995

In [184]:
a = pd.DataFrame({'a': ['1', '2'], 'b': ['3', '4']})
a.at[0, 'a']

'1'

In [266]:
'%.20f' % df[df['project'] == 'ZEPHYR PARK'].at[7959, 'x'], 12

('41180.67381006999494275078', 12)

In [291]:
df1 = pd.read_csv('data/private_prop_transactions/total_transactions.csv')
df1.sort_values('observation_time')

,y,x,street,project,market_segment,area_sqm,area_sqft,price,nett_price,unit_price_psm,unit_price_psf,floor_range,num_units,reference_period,type_of_sale,property_type,district,type_of_area,tenure,observation_time
0,29491.915576,22726.561999,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314.0,3379.867872,4750000.0,NaN,15127.388535,1405.380382,-,1,2018-06-30,Resale,Semi-detached,5,Land,Freehold,2020-04-10 23:05:10.582581
44929,41198.909000,35025.430100,COMPASSVALE ROAD,THE LUXURIE,OCR,98.0,1054.863221,1230000.0,NaN,12551.020408,1166.027951,06-10,1,2017-05-31,Resale,Condominium,19,Strata,99 yrs lease commencing from 2011,2020-04-10 23:05:10.582581
44930,41198.909000,35025.430100,COMPASSVALE ROAD,THE LUXURIE,OCR,87.0,936.460207,913000.0,NaN,10494.252874,974.947994,11-15,1,2018-03-31,Resale,Condominium,19,Strata,99 yrs lease commencing from 2011,2020-04-10 23:05:10.582581
44931,41198.909000,35025.430100,COMPASSVALE ROAD,THE LUXURIE,OCR,72.0,775.001550,930000.0,NaN,12916.666667,1199.997600,06-10,1,2019-04-30,Resale,Condominium,19,Strata,99 yrs lease commencing from 2011,2020-04-10 23:05:10.582581
44932,41198.909000,35025.430100,COMPASSVALE ROAD,THE LUXURIE,OCR,42.0,452.084238,668000.0,NaN,15904.761905,1477.600731,06-10,1,2019-11-30,Resale,Condominium,19,Strata,99 yrs lease commencing from 2011,2020-04-10 23:05:10.582581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67407,32671.914414,34973.777626,IPOH LANE,EMERY POINT,RCR,121.0,1302.433161,725000.0,NaN,5991.735537,556.650446,01-05,1,2017-07-31,Resale,Apartment,15,Strata,NaN,2020-04-10 23:05:38.144093
67404,30747.253000,27457.193100,RIVER VALLEY ROAD,RV RESIDENCES,CCR,63.0,678.126356,1355820.0,NaN,21520.952381,1999.361899,06-10,1,2017-08-31,Resale,Condominium,10,Strata,NaN,2020-04-10 23:05:38.144093
67405,32699.327274,36157.514620,LORONG NANGKA,LANDED HOUSING DEVELOPMENT,RCR,184.4,1984.865081,2750000.0,NaN,14913.232104,1385.484598,-,1,2017-08-31,Resale,Terrace,15,Land,NaN,2020-04-10 23:05:38.144093
67406,36761.646727,29104.810081,CLOVER RISE,LANDED HOUSING DEVELOPMENT,RCR,167.4,1801.878604,2480000.0,NaN,14814.814815,1376.341333,-,1,2017-08-31,Resale,Terrace,20,Land,NaN,2020-04-10 23:05:38.144093


In [292]:
df1[df1['project'] == 'SILVER HILL'].sort_values(['reference_period', 'price'])

,y,x,street,project,market_segment,area_sqm,area_sqft,price,nett_price,unit_price_psm,unit_price_psf,floor_range,num_units,reference_period,type_of_sale,property_type,district,type_of_area,tenure,observation_time
54078,39012.309496,34660.15238,JALAN NAUNG,SILVER HILL,OCR,261.0,2809.380620,2680000.0,NaN,10268.199234,953.946924,-,1,2017-05-31,Resale,Terrace,19,Land,945 yrs lease commencing from 1936,2020-04-10 23:05:10.582581
54079,39012.309496,34660.15238,JALAN NAUNG,SILVER HILL,OCR,152.7,1643.649121,2050000.0,NaN,13425.016372,1247.224833,-,1,2017-08-31,Resale,Terrace,19,Land,NaN,2020-04-10 23:05:10.582581
67408,39012.309496,34660.15238,JALAN NAUNG,SILVER HILL,OCR,152.7,1643.649121,2050000.0,NaN,13425.016372,1247.224833,-,1,2017-08-31,Resale,Terrace,19,Land,NaN,2020-04-10 23:05:38.144093


In [299]:
x = df1[df1['project'] == 'SILVER HILL'].sort_values(['reference_period', 'price']).iloc[[1]].reset_index(drop=True).drop(columns='observation_time')
y = df1[df1['project'] == 'SILVER HILL'].sort_values(['reference_period', 'price']).iloc[[2]].reset_index(drop=True).drop(columns='observation_time')
assert_frame_equal(x, y, check_exact=True, check_index_type=True, check_datetimelike_compat=True, check_categorical=True)


In [306]:
y['tenure']

0    NaN
Name: tenure, dtype: object

In [312]:
df[df['project'] == 'SILVER HILL']

,y,x,street,project,market_segment,area_sqm,area_sqft,price,nett_price,unit_price_psm,unit_price_psf,floor_range,num_units,reference_period,type_of_sale,property_type,district,type_of_area,tenure
26463,39012.309496,34660.15238,JALAN NAUNG,SILVER HILL,OCR,261.0,2809.380620,2680000.0,NaN,10268.199234,953.946924,-,1.0,2017-05-31,Resale,Terrace,19,Land,945 yrs lease commencing from 1936
26464,39012.309496,34660.15238,JALAN NAUNG,SILVER HILL,OCR,152.7,1643.649121,2050000.0,NaN,13425.016372,1247.224833,-,1.0,2017-08-31,Resale,Terrace,19,Land,NA


In [314]:
# for item  in resp.json()['Result']:
#     if item['project'] == 'SILVER HILL':
#         a1 = item
# a1['transaction'][1]
df['tenure'][26464]

'NA'

In [316]:
# pd.DataFrame({'a': []})
subset_cols = ['street', 'project', 'market_segment', 'area_sqm', 'price', 'nett_price', 'floor_range', 'num_units',
               'reference_period', 'type_of_sale', 'property_type', 'district', 'type_of_area', 'tenure']
df11 = df.drop_duplicates(subset=subset_cols)
df11.to_csv('test.csv', index=False)

In [320]:
df1 = pd.read_csv('data/private_prop_transactions/total_transactions.csv')
df1[df1['project'] == 'SILVER HILL']

,y,x,street,project,market_segment,area_sqm,area_sqft,price,nett_price,unit_price_psm,unit_price_psf,floor_range,num_units,reference_period,type_of_sale,property_type,district,type_of_area,tenure,observation_time
54078,39012.309496,34660.15238,JALAN NAUNG,SILVER HILL,OCR,261.0,2809.380620,2680000.0,NaN,10268.199234,953.946924,-,1,2017-05-31,Resale,Terrace,19,Land,945 yrs lease commencing from 1936,2020-04-10 23:24:18.892924
54079,39012.309496,34660.15238,JALAN NAUNG,SILVER HILL,OCR,152.7,1643.649121,2050000.0,NaN,13425.016372,1247.224833,-,1,2017-08-31,Resale,Terrace,19,Land,NaN,2020-04-10 23:24:18.892924


### This is the API call for "Private Residential Property Units Sold by Developers"

In [325]:
headers = {
    "AccessKey": ACCESS_KEY
}
resp = requests.get('https://www.ura.gov.sg/uraDataService/insertNewToken.action', headers=headers)
token = resp.json()['Result']

headers = {
    'AccessKey': ACCESS_KEY,
    'Token': token
}
params = {
    'service': 'PMI_Resi_Developer_Sales',
    'refPeriod': '1019',
}
resp = requests.get('https://www.ura.gov.sg/uraDataService/invokeUraDS', headers=headers, params=params)
resp

<Response [200]>

In [326]:
resp.json()

{'Status': 'Success',
 'Result': [{'street': 'EAST COAST AVENUE',
   'district': '15',
   'propertyType': 'Landed',
   'project': 'CAYMAN RESIDENCES',
   'developer': 'Pinnacle Homes (East Coast) Pte Ltd',
   'developerSales': [{'refPeriod': '1019',
     'medianPrice': 2495,
     'highestPrice': 2606,
     'lowestPrice': 2409,
     'launchedToDate': 19,
     'soldInMonth': 5,
     'launchedInMonth': 0,
     'soldToDate': 17,
     'unitsAvail': 19}],
   'marketSegment': 'OCR'},
  {'street': 'SILAT AVENUE',
   'district': '03',
   'propertyType': 'Non-Landed',
   'project': 'AVENUE SOUTH RESIDENCE',
   'developer': 'United Venture Development (Silat) Pte Ltd',
   'developerSales': [{'refPeriod': '1019',
     'medianPrice': 1990,
     'highestPrice': 2225,
     'lowestPrice': 1802,
     'launchedToDate': 500,
     'soldInMonth': 40,
     'launchedInMonth': 0,
     'soldToDate': 397,
     'unitsAvail': 1074}],
   'marketSegment': 'RCR'},
  {'street': 'DE SOUZA AVENUE',
   'district': '21',